## **Lab 4**

## **Predicting Diabetes Risk Based on BMI**

### **Introduction**

This model is designed to predict the risk of diabetes based on BMI. This is a critical application as diabetes is a widespread and potentially dangerous health condition, and being able to anticipate the risk could lead to early interventions and preventative measures.

### **Train the model**

We will train our model using the scikit-learn's diabetes dataset, which contains real-world clinical data, allowing us to create a robust and reliable prediction model for predicting diabetes risk based on BMI. The dataset's varied variety of variables enables complete model training, supporting the development of a prediction tool capable of accurately assessing diabetes risk in people.


In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

### **Load the Dataset**

In [ ]:
diabetes_ds = datasets.load_diabetes(return_X_y=False, as_frame=False, scaled=False)
X = diabetes_ds.data[:, np.newaxis, 2]
y = diabetes_ds.target